# HDS5210-2020 Midterm

In the midterm, you're going to focus on using the programming skills that you've developed so far to build a calculator for the Apache II scoring system for ICU Mortality.  
* https://www.mdcalc.com/apache-ii-score#evidence
* https://reference.medscape.com/calculator/apache-ii-scoring-system

For the midterm, we'll be building a calculator for the Apache II score and then running that against a patient file that's available to you out on the internet.  This will be broken down into three main steps:
1. Create your JSON file to encapsulate all of the calculation rules for Apache II
2. Create functions to calculate the Apache II score using your JSON configuration
3. Create a function to loop over the patients in a file on the internet and calculate Apach II scores for all of them



---

## Part 1: Creating a JSON Rules File

Look at the rules for the Apache II scoring system on the pages above.  The first step in the midterm is to use those rules and create a JSON configuration file as described in the 2019 midterm video.  I've provided a starter file named `apache.json` to get you started.

Inside that file, you'll find placeholders for all of the measures that go into the Apache II scoring model:
* Organ Failure History
* Age
* Temperature
* [pH](https://en.wikipedia.org/wiki/PH)
* Heart rate
* Respiratory rate
* [Sodium](https://www.mayoclinic.org/diseases-conditions/hyponatremia/symptoms-causes/syc-20373711)
* [Potassium](https://www.emedicinehealth.com/hyperkalemia/article_em.htm)
* [Creatinine](https://www.medicalnewstoday.com/articles/322380)
* [Hematocrit](https://labtestsonline.org/tests/hematocrit)
* White Blood Count
* [FiO2](https://www.ausmed.com/cpd/articles/oxygen-flow-rate-and-fio2)
* [PaO2](https://www.verywellhealth.com/partial-pressure-of-oyxgen-pa02-914920)
* [A-a gradient](https://www.ncbi.nlm.nih.gov/books/NBK545153/)


You may need to create a sort of nested set of rules in some cases.  For instance, the rule for Creatinine says to use certain ranges and points in the case of Acute Renal Failure and a different set of points for Chronic Renal Failure.

Similarly, the rule for FiO2 says to use PaO2 to calculate scores if the FiO2 is <50, and to use A-a Gradient if the PaO2 is >50.

When you've created your `apache.json` file, make sure it's in the same directory as this notebook.

### Testing your JSON

The assert() functions below should all run just fine.  If you want to change the names of any of the keys in the JSON I provided you, you may, but you'll also need to update this test code so that it doesn't fail.  Remember, your notebook should be able to run end-to-end before you submit it.

In [9]:
import json

with open('apache.json') as f:
    rules = json.load(f)

assert('Organ Failure History' in rules.keys())
assert('Age' in rules.keys())
assert('Temperature' in rules.keys())
assert('pH' in rules.keys())
assert('Heart Rate' in rules.keys())
assert('Respiratory Rate' in rules.keys())
assert('Sodium' in rules.keys())
assert('Potassium' in rules.keys())
assert('Creatinine' in rules.keys())
assert('Hematocrit' in rules.keys())
assert('White Blood Count' in rules.keys())
assert('FiO2' in rules.keys())

---

## Part 2: Functions to evaluate rules

Write a series of functions, enough to satisfy all of the main criteria that we're using to calculate the Apache II score.  That list is the same as the assert statements above.

* Each of your functions should be well documented.
* Each function should have "config_file" as one of it's parameters.
* Each function should return a numerical score value.
* Similar to what we discussed in the review, if you can generalize some rules, do so.  You should **NOT** end up with one function for each input variable.  If you did that, you'd have a lot of repetative code.

The Glasgow Coma Scale is simply a 1-to-1 score translation.  Simply add the Glasgow Coma Scale value.  So, you don't need to write a function for this. [Glasgow Coma Scale](https://www.cdc.gov/masstrauma/resources/gcs.pdf)

**CORRECTION ADDED 2/29** - The Glasgow Coma Scale points should be calculated as `1 - Glasgow Coma Scale` rather than what I just stated above.  My preference would be that you do the calculation correctly, as per MDCalc, and then use the **corrected** scores files to compare against as noted in Part 4.

In [65]:
import json
def organ_failure_points(config_file, organ_history):
    with open(config_file) as f:
        config = json.load(f)
    
    points = config.get("Organ Failure History").get(organ_history)
    return points

def creatinine_points(config_file, creatinine, renal_status):
    with open(config_file) as f:
        config = json.load(f)
    points = 99999
    points_rules = config.get("Creatinine").get(renal_status)
    for min_max in points_rules:
        if (  ( float(creatinine) >= float(min_max.get("min")) ) & (  float(creatinine) < float(min_max.get("max")) ) ) :
            points = min_max.get("points")
    return points

def FiO2_points(config_file, FiO2, PaO2, Aa_gradient):
    with open(config_file) as f:
        config = json.load(f)
    
    if FiO2 < 50:
        points_rules = config.get("FiO2").get("PaO2")
        metric = PaO2
    else:
        points_rules = config.get("FiO2").get("A-a gradient")
        metric = Aa_gradient
        
    for min_max in points_rules:
        if (  ( float(metric) >= float(min_max.get("min")) ) & (  float(metric) < float(min_max.get("max")) ) ) :
            points = min_max.get("points")
    return points




In [34]:
organ_failure_points("apache.json", "Nonoperative")

5

In [35]:
def age_points(config_file, age):
    with open(config_file) as f:
        config = json.load(f)
    points = 99
    point_rules = config.get("Age")
    for min_max in point_rules:
        if (  ( float(age) >= float(min_max.get("min")) ) & (  float(age) < float(min_max.get("max")) ) ) :
            points = min_max.get("points")
        
    return points


def temp_points(config_file, temp):
    with open(config_file) as f:
        config = json.load(f)
    points = 99
    point_rules = config.get("Temperature")
    for min_max in point_rules:
        if (  ( float(temp) >= float(min_max.get("min")) ) & (  float(temp) < float(min_max.get("max")) ) ) :
            points = min_max.get("points")
        
    return points

def pH_points(config_file, pH):
    with open(config_file) as f:
        config = json.load(f)
    points = 99
    point_rules = config.get("pH")
    for min_max in point_rules:
        if (  ( float(pH) >= float(min_max.get("min")) ) & (  float(pH) < float(min_max.get("max")) ) ) :
            points = min_max.get("points")
        
    return points

def HR_points(config_file, heartrate):
    with open(config_file) as f:
        config = json.load(f)
    points = 99
    point_rules = config.get("Heart Rate")
    for min_max in point_rules:
        if (  ( float(heartrate) >= float(min_max.get("min")) ) & (  float(heartrate) < float(min_max.get("max")) ) ) :
            points = min_max.get("points")
        
    return points

def RR_points(config_file, respiratory_rate):
    with open(config_file) as f:
        config = json.load(f)
    points = 99
    point_rules = config.get("Respiratory Rate")
    for min_max in point_rules:
        if (  ( float(respiratory_rate) >= float(min_max.get("min")) ) & (  float(respiratory_rate) < float(min_max.get("max")) ) ) :
            points = min_max.get("points")
        
    return points

def sodium_points(config_file, sodium):
    with open(config_file) as f:
        config = json.load(f)
    points = 99
    point_rules = config.get("Sodium")
    for min_max in point_rules:
        if (  ( float(sodium) >= float(min_max.get("min")) ) & (  float(sodium) < float(min_max.get("max")) ) ) :
            points = min_max.get("points")
        
    return points

def potassium_points(config_file, potassium):
    with open(config_file) as f:
        config = json.load(f)
    points = 99
    point_rules = config.get("Potassium")
    for min_max in point_rules:
        if (  ( float(potassium) >= float(min_max.get("min")) ) & (  float(potassium) < float(min_max.get("max")) ) ) :
            points = min_max.get("points")
        
    return points

def hematocrit_points(config_file, hematocrit):
    with open(config_file) as f:
        config = json.load(f)
    points = 99
    point_rules = config.get("Hematocrit")
    for min_max in point_rules:
        if (  ( float(hematocrit) >= float(min_max.get("min")) ) & (  float(hematocrit) < float(min_max.get("max")) ) ) :
            points = min_max.get("points")
        
    return points

def WBC_points(config_file, white_blood_count):
    with open(config_file) as f:
        config = json.load(f)
    points = 99
    point_rules = config.get("White Blood Count")
    for min_max in point_rules:
        if (  ( float(white_blood_count) >= float(min_max.get("min")) ) & (  float(white_blood_count) < float(min_max.get("max")) ) ) :
            points = min_max.get("points")
        
    return points

In [36]:
WBC_points("apache.json", 30)


2

### Testing you Functions

Write enough test cases to verify that your functions work for evaulating all of the scoring inputs.  Have at least 3 test cases for each input.

These tests can be written the same as the assertions we've use in previous assignments.  For example, if you a function for `temperature_score` then you write a test case like:

```
assert( temperature_score(37) == 0 )
```

In [37]:
assert(age_points("apache.json", 50) == 2)

---

## Part 3: Put it all together

Create a new function called `apache_score()` that takes all of the necessary inputs and returns the final Apache II score.  Use any variable names that you want.  For clarity and organization, my recommendation is to create them in the same order as they're documented in the website.

1. Organ Failure History
2. Age
3. Temperature
4. pH 
5. Heart rate
6. Respiratory rate
7. Sodium
8. Potassium
9. Creatinine
10. Acute renal failure
11. Hematocrit
12. White Blood Count
13. Glasgow Coma Scale
14. FiO2
15. PaO2
16. A-a gradient


In [75]:
import csv
import requests


def apache_score(config_file, organ_failure_history, Age, Temp, pH, HR, RR, Sodium, Potassium, Creatinine, arf, hct, wbc, glasgow, fio2, pao2, aa):
    
    """
    apache_score accepts a config_file and the following patient data t return the following apache2 score: Organ Failure History, Age, Temperature, pH, Heart rate, Respiratory rate, Sodium, Potassium, Creatinine, Acute renal failure, Hematocrit, White Blood Count, Glasgow Coma Scale, FiO2, PaO2, A-a gradient
    """
    
    score = age_points(config_file, Age) + organ_failure_points(config_file, organ_failure_history) + temp_points(config_file, Temp) + pH_points(config_file, pH) + HR_points(config_file, HR) + RR_points(config_file, RR) + sodium_points(config_file, Sodium) + potassium_points(config_file, Potassium) + creatinine_points(config_file, Creatinine, arf) + hematocrit_points(config_file, hct) + WBC_points(config_file, wbc) + FiO2_points(config_file, fio2, pao2, aa) + (15 - glasgow)
    return(score)


### Testing your Function

Write a few test cases to make sure that your code functions correctly.  In the last step, you'll have LOTS of test cases run through, but you should do some of your before moving on.

In [39]:
apache_score("apache.json", "Emergency", 6, 27, 7.3, 174, 10, 170,7.8, 1.6, "Chronic Renal Failure", 3, 28, 13, 61, 59, 458)


35

---

## Part 4: Accessing and processing the patient file

Fill out the simple function below to retrieve the patient data as a CSV file from any given URL and return a list of all of the Apache II scores based on the data you find for those patients.
* The patient file will be a CSV
* It will have column headers that match the labels shown above
* The columns will not necessarily appear in the order shown above
* You should output only the Apache II scores, not any other information
* Your output should be a list in the same order as the input rows

In [71]:
def apache_list(config_file, URL):
    """
    apache_list(config_file, URL) outputs a list of integers,
    accepts a filepath to the JSON file including the scoring critera, and the URL pointing to the CSV file of patient data and returns a list of their apache2 scores, 
    CSV file contains the patient data that includes metrics which calculate the apache2 scores,
    The column headings that were used have to match those listed below:
    
    Organ Failure History, Age, Temperature, pH, Heart rate, Respiratory rate, Sodium, Potassium, Creatinine, Acute renal failure, Hematocrit, White Blood Count, Glasgow Coma Scale, FiO2, PaO2, A-a gradient
    """
    
    import pandas as pd
    import json
 
    
    
    data = pd. read_csv(URL)
    df = pd.DataFrame(data)
    score_list = list()
    for patient in df.index:
         score = apache_score(config_file, str(df['Organ Failure History'][patient]), float(df['Age'][patient]), float(df['Temperature'][patient]), float(df['pH'][patient]), float(df['Heart Rate'][patient]), float(df['Respiratory Rate'][patient]), float(df['Sodium'][patient]), float(df['Potassium'][patient]), float(df['Creatinine'][patient]), str(df['Acute Renal Failure'][patient]), float(df['Hematocrit'][patient]), float(df['White Blood Count'][patient]), float(df['Glasgow Coma Scale'][patient]), float(df['FiO2'][patient]), float(df['PaO2'][patient]), float(df['A-a Gradient'][patient]))
         score_list.append(int(score))
         
    return(score_list)

In [72]:
scores = apache_list("apache.json", "https://hds5210-2020.s3.amazonaws.com/TestPatients.csv")
print(scores)

[35, 31, 47, 34, 44, 35, 31, 49, 40, 48, 42, 43, 32, 41, 42, 49, 37, 37, 38, 43, 41, 31, 38, 30, 41, 41, 34, 46, 40, 47, 36, 43, 41, 46, 44, 40, 39, 37, 41, 30, 46, 30, 41, 44, 35, 36, 40, 40, 30, 50, 52, 43, 46, 34, 33, 42, 41, 31, 46, 46, 34, 36, 33, 38, 26, 29, 46, 25, 40, 38, 39, 34, 39, 53, 44, 49, 37, 36, 36, 51, 33, 36, 43, 41, 24, 50, 29, 40, 36, 50, 29, 37, 34, 45, 34, 40, 37, 37, 47, 31, 33, 50, 28, 37, 33, 44, 40, 38, 40, 31, 44, 37, 44, 47, 25, 34, 32, 41, 38, 34, 38, 39, 41, 34, 29, 44, 22, 40, 34, 43, 40, 31, 35, 33, 27, 44, 52, 40, 53, 40, 40, 48, 31, 42, 49, 43, 35, 31, 36, 42, 34, 31, 30, 28, 38, 47, 32, 47, 31, 41, 40, 33, 35, 42, 49, 55, 51, 37, 53, 29, 33, 46, 36, 32, 33, 33, 42, 32, 41, 35, 39, 40, 40, 35, 28, 33, 42, 29, 30, 39, 34, 46, 30, 37, 31, 37, 41, 47, 45, 29, 45, 33, 33, 38, 28, 42, 32, 45, 36, 39, 43, 32, 37, 43, 34, 40, 27, 24, 39, 39, 35, 36, 39, 47, 27, 28, 50, 53, 36, 44, 38, 51, 43, 38, 38, 36, 40, 36, 40, 31, 39, 37, 38, 46, 28, 32, 37, 29, 26, 25,

### Testing your Function

The URL for the test data is: https://hds5210-2020.s3.amazonaws.com/TestPatients.csv


You can verify your results by comparing them against this data: https://hds5210-2020.s3.amazonaws.com/Scores.csv

**CORRECTION ADDED 3/29** - If you calculated the Glasgow Coma Scale points as per the actual instructions in MDCalc, then please use this set of corrected scores to compare your results with: https://hds5210-2020.s3.amazonaws.com/Scores_corrected.csv


In [73]:
import csv
import requests
import codecs


scores = apache_list("apache.json","https://hds5210-2020.s3.amazonaws.com/TestPatients.csv")

check_answers = list()
check_text = requests.get("https://hds5210-2020.s3.amazonaws.com/Scores_corrected.csv").iter_lines()
answers = csv.reader(codecs.iterdecode(check_text, 'utf-8'))

next(answers, None)

for line in answers:
    x = int(float(line[0]))
    check_answers.append(x)


L = max(len(scores),len(check_answers))

print("Whole sequence is equal: ",scores==check_answers)
print("\n------------------------------------------------------- \n")
for i in range(L):
    print(("Calculated score: {}  Check score: {}  Match?:  {}".format(scores[i],check_answers[i],(scores[i]==check_answers[i]))))

Whole sequence is equal:  True

------------------------------------------------------- 

Calculated score: 35  Check score: 35  Match?:  True
Calculated score: 31  Check score: 31  Match?:  True
Calculated score: 47  Check score: 47  Match?:  True
Calculated score: 34  Check score: 34  Match?:  True
Calculated score: 44  Check score: 44  Match?:  True
Calculated score: 35  Check score: 35  Match?:  True
Calculated score: 31  Check score: 31  Match?:  True
Calculated score: 49  Check score: 49  Match?:  True
Calculated score: 40  Check score: 40  Match?:  True
Calculated score: 48  Check score: 48  Match?:  True
Calculated score: 42  Check score: 42  Match?:  True
Calculated score: 43  Check score: 43  Match?:  True
Calculated score: 32  Check score: 32  Match?:  True
Calculated score: 41  Check score: 41  Match?:  True
Calculated score: 42  Check score: 42  Match?:  True
Calculated score: 49  Check score: 49  Match?:  True
Calculated score: 37  Check score: 37  Match?:  True
Calculate